In [39]:
import os
import pandas as pd
import pickle


#exp_name = 'preprocessing_type'
#experiments = ["adam", "other", "rmsprop", "sgd"]
experiments = ["adam", "rmsprop", "sgd"]
grid_search_results = []
for exp in experiments:
    results_path = os.path.join('logs_mac', 'optimizers', exp,'gridsearch_results.pkl')
    with open(results_path, 'rb') as f:
        partial_results = pickle.load(f)
    grid_search_results.extend(partial_results)
print(grid_search_results[0].keys())
    
    

dict_keys(['training-optimizer', 'training-optimizer_params-learning_rate', 'training-optimizer_params-momentum', 'metrics_per_fold'])


In [40]:
print(len(grid_search_results))
for exp in grid_search_results:
    for key in exp.keys():
        if key != 'metrics_per_fold':
            print(f"{key}: {exp[key]}")
    #print(f"epochs: {exp["training-epochs"]}")
    folds = exp["metrics_per_fold"]
    for i, fold in enumerate(folds):
        print(f"\tFold {i+1}:")
        for metric, values in fold.items():
            scaled_val = values["scaled"]
            unscaled_val = values["unscaled"]
            print(f"\t\t{metric}: {scaled_val.metric:.6f} ({unscaled_val.metric:.6f})\n\t\t\tper_sample s (u):\n\t\t\t\t {scaled_val.metric_per_sample}\n\t\t\t\t({unscaled_val.metric_per_sample})")
    print()
    print("######################################")

57
training-optimizer: adam
training-optimizer_params-learning_rate: 0.01
training-optimizer_params-momentum: None
	Fold 1:
		mean_squared_error: 0.000025 (1.111099)
			per_sample s (u):
				 [1.899035e-05, 9.443812e-06, 1.8909755e-05, 9.486262e-06, 4.5606248e-05, 2.1029075e-05, 9.368559e-06, 2.1320537e-05, 1.0650394e-05, 1.3656347e-05, 3.7789254e-05, 4.66918e-05, 9.222036e-06, 1.0823032e-05, 2.1439273e-05, 2.1408043e-05, 1.5442178e-05, 2.1464688e-05, 9.664108e-06, 1.5499856e-05, 9.521674e-06, 1.2582794e-05, 9.568265e-06, 1.39114445e-05, 1.0968672e-05, 8.648104e-05, 4.974492e-05, 2.4678304e-05, 5.3077827e-05, 4.6353947e-05, 3.8188897e-05, 9.440771e-06, 2.1110884e-05, 9.604585e-06, 1.1091267e-05, 6.026208e-05, 1.098803e-05, 9.5654295e-06, 5.827098e-05, 1.0011456e-05, 8.21309e-05, 9.433352e-06, 4.423992e-05, 2.2450515e-05, 9.564896e-06]
				([0.7947531, 0.42102933, 0.76651734, 0.42496, 1.8921056, 1.0273914, 0.40783975, 1.1847293, 0.82418203, 0.50842917, 1.7472838, 1.8973595, 1.0311877, 1

1. 
   - For each experiment, get all samples. 
   - The samples are ordered for each experiment in the same way. 

In [41]:
from sklearn.preprocessing import FunctionTransformer
from types import FunctionType
import re

def get_exp_name(exp):
    name = ""
    for i, key in enumerate(exp.keys()):
        if key != 'metrics_per_fold':
            if i > 0:
                name += " "
            if isinstance(exp[key], str):
                name += exp[key]
            elif isinstance(exp[key], FunctionTransformer):
                return exp[key].func.__name__
            elif isinstance(exp[key], bool):
                name += key if exp[key] else "not "+ key
            elif isinstance(exp[key], FunctionType):
                name += exp[key].__qualname__
            else:
                key_parts = key.split("-")
                last_part = key_parts[-1]
                key_words = last_part.split("_")
                shortcut = "".join([word[0] for word in key_words]) if len(key_words) > 1 else key_words[0]
                name += shortcut + "=" + str(exp[key])

    #name = name.replace("dataset-", "").replace("_", " ").replace("-", " ")
    name = name.capitalize()
    return name

exp_samples = {}
for i, exp in enumerate(grid_search_results):
    exp_name = get_exp_name(exp)
    exp_samples[exp_name] = []
    folds = exp["metrics_per_fold"]
    for fold in folds:
        exp_samples[exp_name].extend(fold["root_mean_squared_error"]["unscaled"].metric_per_sample)
    #print(len(exp_samples[exp_name]))

#for exp_name, samples in exp_samples.items():
#    print(f"{exp_name}: {len(samples)}")
#    print(f"mean: {sum(samples)/len(samples)}")
#    print(f"std: {pd.Series(samples).std()}")
    


In [42]:
from scipy import stats
import statsmodels.stats.multitest
from critdd import Diagram
from itertools import combinations
import numpy as np
import pandas as pd

samples = list(exp_samples.values())
print(np.array(samples).shape)
friedman_result = stats.friedmanchisquare(*samples)
friedman_result

(57, 450)


FriedmanchisquareResult(statistic=24227.641814882023, pvalue=0.0)

In [43]:
combs = combinations(samples, 2)

pvals = []
for comb in combs:
    pvals.append(stats.wilcoxon(comb[0], comb[1]).pvalue)

pvals = np.array(pvals)
pvals.sort()
print(pvals)

for i in range(len(pvals)):
    if pvals[i] > 0.05/(len(pvals)-i):
        print(f"{pvals[i]} is not st. significant")
        print(f"{0.05/(len(pvals)-i)} is the threshold")
        break
    else:
        print(f"{pvals[i]} is st. significant")

statsmodels.stats.multitest.multipletests(pvals, method="holm")

[1.85247108e-75 1.85247108e-75 1.85247108e-75 ... 9.12446587e-01
 9.70089247e-01 9.96964872e-01]
1.852471078383219e-75 is st. significant
1.852471078383219e-75 is st. significant
1.852471078383219e-75 is st. significant
1.852471078383219e-75 is st. significant
1.852471078383219e-75 is st. significant
1.852471078383219e-75 is st. significant
1.852471078383219e-75 is st. significant
1.852471078383219e-75 is st. significant
1.852471078383219e-75 is st. significant
1.852471078383219e-75 is st. significant
1.852471078383219e-75 is st. significant
1.852471078383219e-75 is st. significant
1.8524762278349491e-75 is st. significant
1.8524762278349491e-75 is st. significant
1.8524762278349491e-75 is st. significant
1.8524762278349491e-75 is st. significant
1.8524762278349491e-75 is st. significant
1.8524762278349491e-75 is st. significant
1.8524762278349491e-75 is st. significant
1.8524762278349491e-75 is st. significant
1.8524762278349491e-75 is st. significant
1.8524762278349491e-75 is st. sig

(array([ True,  True,  True, ..., False, False, False]),
 array([2.95654384e-72, 2.95654384e-72, 2.95654384e-72, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00]),
 3.2138139190229786e-05,
 3.132832080200501e-05)

In [44]:
diag_df = pd.DataFrame(exp_samples)
diag_df.columns

Index(['Adam lr=0.01 momentum=none', 'Adam lr=0.001 momentum=none',
       'Adam lr=0.0001 momentum=none', 'Adam lr=1e-05 momentum=none',
       'Nadam lr=0.01 momentum=none', 'Nadam lr=0.001 momentum=none',
       'Nadam lr=0.0001 momentum=none', 'Nadam lr=1e-05 momentum=none',
       'Adamw lr=0.01 momentum=none', 'Adamw lr=0.001 momentum=none',
       'Adamw lr=0.0001 momentum=none', 'Adamw lr=1e-05 momentum=none',
       'Rmsprop lr=0.01 momentum=none', 'Rmsprop lr=0.01 momentum=0.9',
       'Rmsprop lr=0.01 momentum=0.99', 'Rmsprop lr=0.005 momentum=none',
       'Rmsprop lr=0.005 momentum=0.9', 'Rmsprop lr=0.005 momentum=0.99',
       'Rmsprop lr=0.001 momentum=none', 'Rmsprop lr=0.001 momentum=0.9',
       'Rmsprop lr=0.001 momentum=0.99', 'Rmsprop lr=0.0005 momentum=none',
       'Rmsprop lr=0.0005 momentum=0.9', 'Rmsprop lr=0.0005 momentum=0.99',
       'Rmsprop lr=0.0001 momentum=none', 'Rmsprop lr=0.0001 momentum=0.9',
       'Rmsprop lr=0.0001 momentum=0.99', 'Sgd lr=0.01 m

In [45]:
exp_samples.values()

dict_values([[0.89148986, 0.64886624, 0.8755114, 0.6518883, 1.3755352, 1.0136031, 0.6386234, 1.0884548, 0.9078449, 0.71304005, 1.3218417, 1.3774447, 1.0154738, 1.03831, 1.0935938, 1.0936654, 0.75813955, 1.0759076, 0.6762582, 0.7570177, 1.0198579, 0.7068066, 0.6579652, 0.71940297, 1.0409355, 1.6839207, 1.3915688, 1.2042539, 1.4175571, 1.3774568, 1.3188926, 0.646269, 1.0312529, 0.74610263, 0.94999313, 1.4292061, 0.93723655, 0.73462886, 1.4524562, 0.6639346, 1.6061075, 0.63136965, 1.3720328, 1.0714304, 0.6605602, 0.82793653, 0.9101624, 0.7215244, 1.4781873, 0.6420735, 0.94604707, 1.6671304, 0.64595497, 0.7850894, 1.6807854, 1.656027, 1.6493295, 0.79950565, 0.6658071, 1.4898763, 0.8999245, 0.7579739, 0.6730733, 1.6283337, 1.4170097, 1.47639, 0.63319933, 1.5651507, 0.7916155, 1.6632012, 1.5691298, 1.0683614, 1.5033203, 1.496931, 1.4987003, 0.81270516, 0.90500146, 0.8141662, 0.8696352, 0.8484611, 0.8304256, 0.86667883, 0.92093986, 0.92678976, 1.1763614, 0.8125406, 0.6557024, 0.63890827, 0.68

In [51]:
x=np.array(list(exp_samples.values()))
treatment_names = list(exp_samples.keys())
print(treatment_names)
x = x.T
diagram = Diagram(x,
    treatment_names=treatment_names,
    maximize_outcome = False,
)

['Adam lr=0.01 momentum=none', 'Adam lr=0.001 momentum=none', 'Adam lr=0.0001 momentum=none', 'Adam lr=1e-05 momentum=none', 'Nadam lr=0.01 momentum=none', 'Nadam lr=0.001 momentum=none', 'Nadam lr=0.0001 momentum=none', 'Nadam lr=1e-05 momentum=none', 'Adamw lr=0.01 momentum=none', 'Adamw lr=0.001 momentum=none', 'Adamw lr=0.0001 momentum=none', 'Adamw lr=1e-05 momentum=none', 'Rmsprop lr=0.01 momentum=none', 'Rmsprop lr=0.01 momentum=0.9', 'Rmsprop lr=0.01 momentum=0.99', 'Rmsprop lr=0.005 momentum=none', 'Rmsprop lr=0.005 momentum=0.9', 'Rmsprop lr=0.005 momentum=0.99', 'Rmsprop lr=0.001 momentum=none', 'Rmsprop lr=0.001 momentum=0.9', 'Rmsprop lr=0.001 momentum=0.99', 'Rmsprop lr=0.0005 momentum=none', 'Rmsprop lr=0.0005 momentum=0.9', 'Rmsprop lr=0.0005 momentum=0.99', 'Rmsprop lr=0.0001 momentum=none', 'Rmsprop lr=0.0001 momentum=0.9', 'Rmsprop lr=0.0001 momentum=0.99', 'Sgd lr=0.01 momentum=none', 'Sgd lr=0.01 momentum=0.9', 'Sgd lr=0.01 momentum=0.99', 'Sgd lr=0.005 momentum=no

In [52]:
def get_best_optimizer_by_type(diagram: Diagram):
    """Gets the best optimizer for each type (Adam, RMSprop, SGD, ...)"""
    opt_types = set([name.split(" ")[0] for name in treatment_names])
    print(opt_types)
    sorted_treatments = sorted(zip(diagram.treatment_names, diagram.average_ranks), key=lambda x: x[1])
    best_optimizers = {}
    for name, avg_rank in sorted_treatments:
        opt_type = name.split(" ")[0]
        if opt_type not in best_optimizers:
            best_optimizers[opt_type] = (name, avg_rank)
    return best_optimizers

def get_partial_diagram(subset_names, samples):
    x = np.array([samples[name] for name in subset_names])
    x = x.T
    return Diagram(x,
        treatment_names=subset_names,
        maximize_outcome = False,
    )

best_optimizers = get_best_optimizer_by_type(diagram)
diagram = get_partial_diagram([name for name, _ in best_optimizers.values()], exp_samples)

print("Previous average ranks:")
for name, avg_r in best_optimizers.values():
    print(f"{name}: {avg_r}")

print("\nNew average ranks:")
sorted_treatments = sorted(zip(diagram.treatment_names, diagram.average_ranks), key=lambda x: x[1])
for name, avg_r in sorted_treatments:
    print(f"{name}: {avg_r}")

diagram.to_file(
    "critdd_optimizers_best_per_type.tex",
    alpha=0.05,
    adjustment="holm",
    reverse_x=True,
)

{'Rmsprop', 'Nadam', 'Adam', 'Sgd_nesterov', 'Sgd', 'Adamw'}
Previous average ranks:
Rmsprop lr=0.0001 momentum=0.99: 3.2488888888888887
Adamw lr=0.01 momentum=none: 4.602222222222222
Adam lr=0.01 momentum=none: 4.68
Nadam lr=0.01 momentum=none: 8.24
Sgd_nesterov lr=0.01 momentum=0.99: 13.597777777777777
Sgd lr=0.01 momentum=0.99: 16.342222222222222

New average ranks:
Rmsprop lr=0.0001 momentum=0.99: 2.14
Adam lr=0.01 momentum=none: 2.4266666666666667
Adamw lr=0.01 momentum=none: 2.533333333333333
Nadam lr=0.01 momentum=none: 3.511111111111111
Sgd_nesterov lr=0.01 momentum=0.99: 4.848888888888889
Sgd lr=0.01 momentum=0.99: 5.54
